## Machines Manufacturing Captal Budgeting Model


- [**Overview**](#Overview):: Brief description of the problem and the model
- [**Setup**](#Setup): Runs any imports and other setup
- [**Inputs**](#Inputs): Defines the inputs for the model
- [**Machines Output**](#machine-output): Determining the output of a machine and the output for multiple machines
- [**Demand**](#demand): Determines the number of units demanded per year, based on the number of advertising years
- [**Cash Flows**](#cash-flows): Combines the machines output and demand to determine the quantity transacted each year. Then uses this information as well as phone prices and costs, machine scrap values, and machine/advertising costs to determine the cash flows in each year.
- [**NPV**](#npv): Calculates a net present value (NPV) from the cash flows.
- [**Final Outputs**](#final-outputs): Shows the final output cash flows and NPV with formating.

## Overview


### The Problem

You work for a new startup that is trying to manufacture phones. You are tasked with building a model which will help determine how many machines to invest in and how much to spend on marketing. Each machine produces $n_{output}$ phones per year. Each phone sells for $P_{phone}$ and costs $c_{phone}$ in variable costs to produce. After  years, the machine can no longer produce output, but may be scrapped for $p_{scrap}$. The machine will not be replaced, so you may end up with zero total output before your model time period ends.

Equity investment is limited, so in each year you can spend $c_{machine}$ to either buy a machine or buy advertisements. In the first year you must buy a machine. Any other machine purchases must be made one after another (advertising can only begin after machine buying is done). Demand for your phones starts at $d_1$. Each time you advertise, demand increases by $g_d$%. The prevailing market interest rate is $r$.

**Notes**

- Model is limited to 20 years and a maximum of 5 machines.
- For simplicity, assume that  is paid in every year, even after all machines have shut down.
- Ensure that you can change the inputs and the outputs change as expected.
- For simplicity, assume that fractional phones can be sold, you do not need to round the quantity transacted.

## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [35]:
from dataclasses import dataclass
import numpy_financial as npf
import itertools

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [20]:
@dataclass
class ModelInputs:
    n_phones: float = 100000
    price_scrap: float = 50000
    price_phone: float = 500
    cost_machine_adv: float = 1000000
    costs_phone: float = 250
    n_life: int = 10
    n_machines: int = 5
    d_1: float = 100000
    g_d: float = 0.2
    max_year: float = 20
    interest: float = 0.05
        
    # Inputs for bonus problem
    elasticity: float = 100
    demand_constant: float = 300000
        
model_data = ModelInputs()
data = model_data
model_data

ModelInputs(n_phones=100000, price_scrap=50000, price_phone=500, cost_machine_adv=1000000, costs_phone=250, n_life=10, n_machines=5, d_1=100000, g_d=0.2, max_year=20, interest=0.05, elasticity=100, demand_constant=300000)

We will create several functions to solve this problem, this will help to solve similar projects with little changes instead of having to hard-code each time we want to solve the project. This will allow us to add more analysis later for the project.

## Machine Outputs

The first set of functions will be related to the enterprise machines.

They will serve to determine:

- Machines bought per year
- Number of working machines per year
- Broken machines per year
- Scrap value of a broken machine per year

In [21]:
def machines_bought_per_year(data: ModelInputs):
    '''
    Creates a list of booleans when a machine is bought, it will be true for the first years according 
    to the number of machines up to the last year of the model.
    '''
    years = list(range(1, data.max_year + 1))
    buy_machine = []
    for year in years:
            # This will create a list of booleans when a machine is bought
            if year <= data.n_machines:
                buy_machine.append(True)
            else:
                buy_machine.append(False)
    return buy_machine

In [22]:
def broken_machines_per_year(data: ModelInputs):
    '''
    Creates a list of booleans when a machine breakes down, it will be true after each machine bought
    ends the years of life (given in model inputs)
    '''
    # This creates a list of booleans when a machine breaks down
    years = list(range(1, data.max_year + 1))
    broken_machine = []
    for year in years:
        if data.n_life % year == data.n_life and year <= (data.n_machines + data.n_life):
            broken_machine.append(True)
        else:
            broken_machine.append(False)
    return broken_machine

In [23]:
def working_machines_per_year():
    '''
    This will create a list of ints when a machine is bought, when a machine
    breaks down and when none of these happens at the same time (1, -1, 0), respectevely
    '''
    working_machine = []
    machines_bought = machines_bought_per_year(model_data)
    broken_machines = broken_machines_per_year(model_data)
    for i, j in zip(machines_bought, broken_machines):
        if i == True and j == False:
            working_machine.append(1)
        elif i == False and j == True:
            working_machine.append(-1)
        else:
            working_machine.append(0)
    working_machine = list(itertools.accumulate(working_machine))
    return working_machine

In [24]:
def scrap_value_per_year(data: ModelInputs):
    '''
    This function returns the scrap value per year, will only return a value when a machine
    breakes down
    '''
    scrap_value = []
    broken_machines = broken_machines_per_year(data)
    for i in broken_machines:
        if i == True:
            scrap_value.append(data.price_scrap)
        elif i == False:
            scrap_value.append(0)
    return scrap_value

## Demand

Now that we have the information for all the machines, we can start to look at the demand of phones, which will increase after all machines are bought since the company will start to invest in advertisement.

With that in mind, we can get the number of phones sold per year, recall that for simplicity of this project, fractional phones can be sold.

In [25]:
def demand_of_phones(data: ModelInputs):
    '''
    This function creates a list of the demanded phones, will start increasing after all machines are bought
    '''
    demand = []
    machines_bought = machines_bought_per_year(data)
    for i in machines_bought:
        if i == True:
            demand.append(data.d_1)
        elif i == False:
            demand.append(demand[-1] * (1 + data.g_d))
    return demand

In [26]:
def phones_sold_per_year(data: ModelInputs):
    '''
    This function creates the number of phones produced depending on the number
    of working machines, then it returns phones sold according to the demand 
    per year
    '''
    working_machines = working_machines_per_year()
    phones_produced = [i * data.n_phones for i in working_machines]
    demand = demand_of_phones(data)
    phones_sold = []
    for p, d in zip(phones_produced, demand):
        if p >= d:
            phones_sold.append(d)
        else:
            phones_sold.append(p)
    return phones_sold

Numer of phones sold per year

In [27]:
phones_sold_per_year(model_data)

[100000,
 100000,
 100000,
 100000,
 100000,
 120000.0,
 144000.0,
 172800.0,
 207360.0,
 248832.0,
 298598.39999999997,
 300000,
 200000,
 100000,
 0,
 0,
 0,
 0,
 0,
 0]

## Cash Flows

We now have all the necessary information to get the cash flows for the company

In [30]:
def cash_flows_per_year(data: ModelInputs):  
    '''
    This function takes the inputs of the class and returns
    the cash flows of the project
    '''
    revenues = []
    money_in = []
    cogs = []
    money_out = []
    phones = phones_sold_per_year(data)
    scrap = scrap_value_per_year(data)
    
    # This creates a list of revenues, which multiplies each number of phones
    # sold by year by the price given in the class
    revenues = [i * data.price_phone for i in phones]
    
    # This creates a list of cogs, which multiplies each number of phones
    # sold by year by the variable cost per phone given in the class
    cogs = [i * data.costs_phone for i in phones]
    
    # This creates a list of the revenues plus the scrap value of the machines per year
    money_in = [x + y for x, y in zip(revenues, scrap)]
    
    # This creates a list of cogs plus cash spent per year
    money_out = [i + data.cost_machine_adv for i in cogs]
    
    # The money-in minus the money-out of each year will be our cash flow
    cash_flows = [x - y for x, y in zip(money_in, money_out)]
    return cash_flows

In [31]:
cash_flows_per_year(model_data)

[24000000,
 24000000,
 24000000,
 24000000,
 24000000,
 29000000.0,
 35000000.0,
 42200000.0,
 50840000.0,
 61208000.0,
 73699599.99999999,
 74050000,
 49050000,
 24050000,
 -950000,
 -1000000,
 -1000000,
 -1000000,
 -1000000,
 -1000000]

## NPV

This section determines the NPV of the project with the current inputs. This is the main output from the model.

In [32]:
def model_npv(data: ModelInputs):
    '''
    Determines the NPV of the entire phone manufacturing capital budget problem.
    Uses the cash flows from cash_flows_per_year and calculates the NPV from them
    '''
    # numpy NPV assumes first year is zero, we don't have a year zero so add a zero cash flow
    cf = cash_flows_per_year(data)
    all_cash_flows = [0] + cf
    npv = npf.npv(data.interest, all_cash_flows)
    return npv

In [33]:
npv = model_npv(model_data)
npv

369276542.47415346

## Final Outputs

We have calculated all the cash flows for the years and the corresponding net present value, just to make it more readable, we will format the answers.

In [34]:
formatted_cash_flows = [f'Year {i + 1}: ${cf:,.0f}' for i, cf in enumerate(cash_flows_per_year(model_data))]
print('Cash Flows:\n' + '\n'.join(formatted_cash_flows))
print(f'\n\nNPV: ${npv:,.0f}')

Cash Flows:
Year 1: $24,000,000
Year 2: $24,000,000
Year 3: $24,000,000
Year 4: $24,000,000
Year 5: $24,000,000
Year 6: $29,000,000
Year 7: $35,000,000
Year 8: $42,200,000
Year 9: $50,840,000
Year 10: $61,208,000
Year 11: $73,699,600
Year 12: $74,050,000
Year 13: $49,050,000
Year 14: $24,050,000
Year 15: $-950,000
Year 16: $-1,000,000
Year 17: $-1,000,000
Year 18: $-1,000,000
Year 19: $-1,000,000
Year 20: $-1,000,000


NPV: $369,276,542
